In [31]:
import pandas as pd
import numpy as np
from pprint import pprint
import time

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
nltk.download('all')

from bs4 import BeautifulSoup
import requests
import os

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/ramirolopez/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/ramirolopez/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/ramirolopez/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/ramirolopez/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/ramirolopez/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |     /Users/

In [2]:
base_url = 'https://codeup.edu/blog/'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0"}
codeup_request = requests.get(base_url, headers=headers)
soup = BeautifulSoup(codeup_request.text, 'html.parser')
blog_links = [element['href'] for element in soup.find_all('a', class_='more-link')]

In [3]:
# lets iterate through those blog links an build out a structure
# that will allow me to scrape the contents of that page
all_blogs = []
for link in blog_links:
    # get a response from the link element
    response = requests.get(link, headers={'User-Agent': 'Robinson Rulez lol'})
    #turn the response text into a soup
    soup = BeautifulSoup(response.text, 'html.parser')
    #grab the html element associated with the article body
    title = soup.find('h1', class_='entry-title').text
    #grab the html element associated with the article head
    body = soup.find('div', class_='entry-content').text.strip()
    # toss those two things with labels into a dictionary
    row = {'title': title, 'article': body}
    #add that dictionary to a list of dictionaries
    all_blogs.append(row)
#outside the loop:
# cast the list of dictionaries into a pandas DataFrame
articles = pd.DataFrame(all_blogs)

In [46]:
articles

,title,article
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight – Sarah Mell...
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight – Madeleine ...
5,Black Excellence in Tech: Panelist Spotlight –...,Black excellence in tech: Panelist Spotlight –...


In [5]:
base_url = 'https://inshorts.com/en/read/'
categories = [
    'business',
    'entertainment',
    'technology',
    'sports'
]
test_soup = BeautifulSoup(requests.get(base_url+categories[0]).text, 'html.parser')
time.time()

1697575846.3459892

In [6]:
all_articles = pd.DataFrame(columns=['title', 'body','category'])
t_sum = 0
for category in categories:
    t_0 = time.time()
    print(f'Grabbing contents for {category}.')
    # construct a url based on our base concatenated with the cat
    category_url = base_url + category
    # get the response text from the constructed url
    raw_content = requests.get(category_url).text
    #turn the content into soup:
    soup = BeautifulSoup(raw_content, 'html.parser')
    # title content:
    titles = [
        element.text for element in soup.find_all(
            'span', itemprop='headline')
    ]
    # body content:
    bodies = [
        element.text for element in soup.find_all(
            'div', itemprop='articleBody')
    ]
    # category (its already here :) ) 
    category_df = pd.DataFrame(
    {
        'title': titles,
        'body': bodies,
        'category': category
    })
    all_articles = pd.concat(
    [
        all_articles,
        category_df
    ],
    axis=0,
    ignore_index=True)
    t_n = time.time()
    t_delta = t_n - t_0
    print(f'Time to grab contents of {category}: {round(t_delta, 2)} seconds')
    t_sum += t_delta
print('Job finished!')
print(f'It took {round((t_sum / 60), 2)} minutes to execute scraping')

Grabbing contents for business.
Time to grab contents of business: 0.18 seconds
Grabbing contents for entertainment.
Time to grab contents of entertainment: 0.08 seconds
Grabbing contents for technology.
Time to grab contents of technology: 0.09 seconds
Grabbing contents for sports.
Time to grab contents of sports: 0.17 seconds
Job finished!
It took 0.01 minutes to execute scraping


In [7]:
all_articles

,title,body,category
0,Victoria's Secret ex-CEO cuts Harvard ties for...,Victoria's Secret ex-CEO Leslie Wexner's found...,business
1,HDFC Bank's Vigil Aunty ad gets criticism for ...,HDFC Bank's latest advertisement featuring Vig...,business
2,IMEC big opportunity for investors to partner ...,PM Narendra Modi at the Global Maritime India ...,business
3,"ICICI Bank fined ₹12 crore, Kotak ₹3.95 crore ...",The Reserve Bank of India (RBI) has imposed a ...,business
4,35 lakh weddings in 23 days to generate record...,Traders' body Confederation of All India Trade...,business
5,"Mahadev app key accused arrives from Dubai, he...","Mrugank Mishra, a key accused in Mahadev betti...",business
6,Bankman-Fried's trial delay request over Adder...,A US court denied FTX Founder Sam Bankman-Frie...,business
7,Indian wheat prices hit 8-month high,Prices of wheat in India have reached an eight...,business
8,Dabur gets ₹321 crore GST demand notice,Dabur India has received a notice to pay Goods...,business
9,'Kill list' of LinkedIn staff being fired was ...,A list with names of about 500 employees was l...,business


# Q1
## Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:

- Lowercase everything
- Normalize unicode characters
- Replace anything that is not a letter, number, whitespace or a single quote.


In [8]:
test_string = 'lopez.j.ramiro1@gmail.com is my - Email. (678)937-1910 is my Phone #.'

In [9]:
# made everything lowercase
test_string = test_string.lower()
print(test_string)

lopez.j.ramiro1@gmail.com is my - email. (678)937-1910 is my phone #.


In [10]:
# replaced anything that is not a letter, number, whitespace or single quite. 
test_string = re.sub(r"[^a-z0-9'\s]", '', test_string)
print(test_string)

lopezjramiro1gmailcom is my  email 6789371910 is my phone 


In [11]:
# normalize unicode characters 
test_string = unicodedata.normalize('NFKD', test_string)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')

print(test_string)


lopezjramiro1gmailcom is my  email 6789371910 is my phone 


In [12]:
def basic_clean(paragraph):
    paragraph = paragraph.lower()
    paragraph = re.sub(r"[^a-z0-9'\s]", '', paragraph)
    paragraph = unicodedata.normalize('NFKD', paragraph)\
    .encode('ascii', 'ignore')\
    .decode('utf-8', 'ignore')
    return paragraph

In [13]:
# test on code up blog article 
sep_line = "=" * 80
for art in articles.article[:2]:
    pprint(basic_clean(art))
    print(' ')
    print(sep_line)
    print(' ')

('may is traditionally known as asian american and pacific islander aapi '
 'heritage month this month we celebrate the history and contributions made '
 'possible by our aapi friends family and community we also examine our level '
 'of support and seek opportunities to better understand the aapi community\n'
 '\n'
 'in an effort to address real concerns and experiences we sat down with '
 'arbeena thapa one of codeups financial aid and enrollment managers\n'
 'arbeena identifies as nepali american and desi arbeenas parents immigrated '
 'to texas in 1988 for better employment and educational opportunities '
 'arbeenas older sister was five when they made the move to the us arbeena was '
 'born later becoming the first in her family to be a us citizen\n'
 'at codeup we take our efforts at inclusivity very seriously after speaking '
 'with arbeena we were taught that the term aapi excludes desiamerican '
 'individuals hence we will now use the term asian pacific islander desi '
 'ameri

# Q2
## Define a function named tokenize. It should take in a string and tokenize all the words in the string.

In [14]:
tokenizer = nltk.tokenize.ToktokTokenizer()

print(tokenizer.tokenize(articles.article[0], return_str=True)[0:500])


May is traditionally known as Asian American and Pacific Islander ( AAPI ) Heritage Month. This month we celebrate the history and contributions made possible by our AAPI friends , family , and community. We also examine our level of support and seek opportunities to better understand the AAPI community.

In an effort to address real concerns and experiences , we sat down with Arbeena Thapa , one of Codeup ’ s Financial Aid and Enrollment Managers.
Arbeena identifies as Nepali American and Desi.


In [15]:
def tokenize(paragraph):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(paragraph, return_str=True)

In [16]:
tokenize(articles.article[4])

'Women in tech : Panelist Spotlight – Madeleine Capper\nCodeup is hosting a Women in Tech Panel in honor of Women ’ s History Month on March 29th , 2023 ! To further celebrate , we ’ d like to spotlight each of our panelists leading up to the discussion to learn a bit about their respective experiences as women in the tech industry ! \nMeet Madeleine ! \nMadeleine Capper is a Data Scientist in San Antonio , Texas. A long-standing San Antonio resident , she studied mathematics at the University of Texas San Antonio and has worked as a Data Scientist for Booz Allen Hamilton. Madeleine currently teaches Data Science at Codeup , where she works daily with burgeoning data professionals to help them actualize their career aspirations through technical education.\nMadeleine attended Codeup as a student in early 2019 as a pupil in the very first Codeup Data Science cohort. The program proved immediately effective and she was the first student to obtain a data career out of the program. After w

# Q3
### Define a function named stem. It should accept some text and return the text after applying stemming to all the words.


In [17]:
# Create the nltk stemmer object, then use it
ps = nltk.porter.PorterStemmer()

In [18]:
stems = [ps.stem(word) for word in articles.article[0].split()]
article_stemmed = ' '.join(stems)
print(article_stemmed)


may is tradit known as asian american and pacif island (aapi) heritag month. thi month we celebr the histori and contribut made possibl by our aapi friends, family, and community. we also examin our level of support and seek opportun to better understand the aapi community. in an effort to address real concern and experiences, we sat down with arbeena thapa, one of codeup’ financi aid and enrol managers. arbeena identifi as nepali american and desi. arbeena’ parent immigr to texa in 1988 for better employ and educ opportunities. arbeena’ older sister wa five when they made the move to the us. arbeena wa born later, becom the first in her famili to be a us citizen. at codeup we take our effort at inclus veri seriously. after speak with arbeena, we were taught that the term aapi exclud desi-american individuals. hence, we will now use the term asian pacif island desi american (apida). here is how the rest of our convers with arbeena went! how do you celebr or connect with your heritag an

In [19]:
pd.Series(stems).value_counts().head(10)


the      29
and      27
to       24
as       17
in       17
of       17
a        16
is       11
for      11
asian    11
Name: count, dtype: int64

In [20]:
def stem(paragraph):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in paragraph.split()]
    paragraph_stemmed = ' '.join(stems)
    return paragraph_stemmed
    

In [21]:
art_one = stem(articles.article[0])

# Q4
### Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.


In [29]:
test_string = articles.article[0]

In [32]:
wnl = nltk.stem.WordNetLemmatizer()


for word in test_string.split():
    print(wnl.lemmatize(word))


May
is
traditionally
known
a
Asian
American
and
Pacific
Islander
(AAPI)
Heritage
Month.
This
month
we
celebrate
the
history
and
contribution
made
possible
by
our
AAPI
friends,
family,
and
community.
We
also
examine
our
level
of
support
and
seek
opportunity
to
better
understand
the
AAPI
community.
In
an
effort
to
address
real
concern
and
experiences,
we
sat
down
with
Arbeena
Thapa,
one
of
Codeup’s
Financial
Aid
and
Enrollment
Managers.
Arbeena
identifies
a
Nepali
American
and
Desi.
Arbeena’s
parent
immigrated
to
Texas
in
1988
for
better
employment
and
educational
opportunities.
Arbeena’s
older
sister
wa
five
when
they
made
the
move
to
the
US.
Arbeena
wa
born
later,
becoming
the
first
in
her
family
to
be
a
US
citizen.
At
Codeup
we
take
our
effort
at
inclusivity
very
seriously.
After
speaking
with
Arbeena,
we
were
taught
that
the
term
AAPI
excludes
Desi-American
individuals.
Hence,
we
will
now
use
the
term
Asian
Pacific
Islander
Desi
American
(APIDA).
Here
is
how
the
rest
of
our
conversat

In [34]:
lemmas = [wnl.lemmatize(word) for word in test_string.split()]
article_lemmatized = ' '.join(lemmas)

print(article_lemmatized)

pd.Series(lemmas).value_counts()[:10]


May is traditionally known a Asian American and Pacific Islander (AAPI) Heritage Month. This month we celebrate the history and contribution made possible by our AAPI friends, family, and community. We also examine our level of support and seek opportunity to better understand the AAPI community. In an effort to address real concern and experiences, we sat down with Arbeena Thapa, one of Codeup’s Financial Aid and Enrollment Managers. Arbeena identifies a Nepali American and Desi. Arbeena’s parent immigrated to Texas in 1988 for better employment and educational opportunities. Arbeena’s older sister wa five when they made the move to the US. Arbeena wa born later, becoming the first in her family to be a US citizen. At Codeup we take our effort at inclusivity very seriously. After speaking with Arbeena, we were taught that the term AAPI excludes Desi-American individuals. Hence, we will now use the term Asian Pacific Islander Desi American (APIDA). Here is how the rest of our conversat

a      31
the    29
and    27
to     23
of     17
in     15
for    11
is     11
you    11
or     10
Name: count, dtype: int64

In [53]:
def lemmonize(paragraph):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in paragraph.split()]
    article_lemmatized = ' '.join(lemmas)
    return article_lemmatized
    
    

In [54]:
lemmonize(articles.article[1])

'Women in tech: Panelist Spotlight – Magdalena Rahn Codeup is hosting a Women in Tech Panel in honor of Women’s History Month on March 29th, 2023! To further celebrate, we’d like to spotlight each of our panelist leading up to the discussion to learn a bit about their respective experience a woman in the tech industry! Meet Magdalena! Magdalena Rahn is a current Codeup student in a Data Science cohort in San Antonio, Texas. She ha a professional background in cross-cultural communications, international business development, the wine industry and journalism. After serving in the US Navy, she decided to complement her professional skill set by attending the Data Science program at Codeup; she is set to graduate in March 2023. Magdalena is fluent in French, Bulgarian, Chinese-Mandarin, Spanish and Italian. We asked Magdalena how Codeup impacted her career, and she replied “Codeup ha provided a solid foundation in analytical processes, programming and data science methods, and it’s been a

# Q5
### Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.

- This function should define two optional parameters, extra_words and exclude_words. These parameters should define any additional stop words to include, and any words that we don't want to remove.


In [41]:
stopword_list = stopwords.words('english')

stopword_list.remove('no')
stopword_list.remove('not')

stopword_list[:10]


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [42]:
words = test_string.split()
filtered_words = [w for w in words if w not in stopword_list]

print('Removed {} stopwords'.format(len(words) - len(filtered_words)))
print('---')

article_without_stopwords = ' '.join(filtered_words)

print(article_without_stopwords)


Removed 333 stopwords
---
May traditionally known Asian American Pacific Islander (AAPI) Heritage Month. This month celebrate history contributions made possible AAPI friends, family, community. We also examine level support seek opportunities better understand AAPI community. In effort address real concerns experiences, sat Arbeena Thapa, one Codeup’s Financial Aid Enrollment Managers. Arbeena identifies Nepali American Desi. Arbeena’s parents immigrated Texas 1988 better employment educational opportunities. Arbeena’s older sister five made move US. Arbeena born later, becoming first family US citizen. At Codeup take efforts inclusivity seriously. After speaking Arbeena, taught term AAPI excludes Desi-American individuals. Hence, use term Asian Pacific Islander Desi American (APIDA). Here rest conversation Arbeena went! How celebrate connect heritage cultural traditions? “I celebrate Nepal’s version Christmas Dashain. This nine-day celebration also known Dussehra. I grew Hindu I iden

Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe news_df.

In [57]:
all_articles.head()

,title,body,category
0,Victoria's Secret ex-CEO cuts Harvard ties for...,Victoria's Secret ex-CEO Leslie Wexner's found...,business
1,HDFC Bank's Vigil Aunty ad gets criticism for ...,HDFC Bank's latest advertisement featuring Vig...,business
2,IMEC big opportunity for investors to partner ...,PM Narendra Modi at the Global Maritime India ...,business
3,"ICICI Bank fined ₹12 crore, Kotak ₹3.95 crore ...",The Reserve Bank of India (RBI) has imposed a ...,business
4,35 lakh weddings in 23 days to generate record...,Traders' body Confederation of All India Trade...,business


In [59]:
all_articles['clean'] = all_articles['body'].apply(basic_clean)

In [60]:
all_articles.head(3)

,title,body,category,clean
0,Victoria's Secret ex-CEO cuts Harvard ties for...,Victoria's Secret ex-CEO Leslie Wexner's found...,business,victoria's secret exceo leslie wexner's founda...
1,HDFC Bank's Vigil Aunty ad gets criticism for ...,HDFC Bank's latest advertisement featuring Vig...,business,hdfc bank's latest advertisement featuring vig...
2,IMEC big opportunity for investors to partner ...,PM Narendra Modi at the Global Maritime India ...,business,pm narendra modi at the global maritime india ...


In [61]:
all_articles['stemmed'] =all_articles['body'].apply(stem)

In [62]:
all_articles.head(3)

,title,body,category,clean,stemmed
0,Victoria's Secret ex-CEO cuts Harvard ties for...,Victoria's Secret ex-CEO Leslie Wexner's found...,business,victoria's secret exceo leslie wexner's founda...,victoria' secret ex-ceo lesli wexner' foundat ...
1,HDFC Bank's Vigil Aunty ad gets criticism for ...,HDFC Bank's latest advertisement featuring Vig...,business,hdfc bank's latest advertisement featuring vig...,hdfc bank' latest advertis featur vigil aunti ...
2,IMEC big opportunity for investors to partner ...,PM Narendra Modi at the Global Maritime India ...,business,pm narendra modi at the global maritime india ...,pm narendra modi at the global maritim india s...


In [63]:
all_articles['lemmatizatized'] = all_articles['body'].apply(lemmonize)

In [64]:
all_articles.head(3)

,title,body,category,clean,stemmed,lemmatizatized
0,Victoria's Secret ex-CEO cuts Harvard ties for...,Victoria's Secret ex-CEO Leslie Wexner's found...,business,victoria's secret exceo leslie wexner's founda...,victoria' secret ex-ceo lesli wexner' foundat ...,Victoria's Secret ex-CEO Leslie Wexner's found...
1,HDFC Bank's Vigil Aunty ad gets criticism for ...,HDFC Bank's latest advertisement featuring Vig...,business,hdfc bank's latest advertisement featuring vig...,hdfc bank' latest advertis featur vigil aunti ...,HDFC Bank's latest advertisement featuring Vig...
2,IMEC big opportunity for investors to partner ...,PM Narendra Modi at the Global Maritime India ...,business,pm narendra modi at the global maritime india ...,pm narendra modi at the global maritim india s...,PM Narendra Modi at the Global Maritime India ...


Make another dataframe for the Codeup blog posts. Name the dataframe codeup_df.

In [47]:
articles.head()

,title,article
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...
3,Women in Tech: Panelist Spotlight – Sarah Mellor,Women in tech: Panelist Spotlight – Sarah Mell...
4,Women in Tech: Panelist Spotlight – Madeleine ...,Women in tech: Panelist Spotlight – Madeleine ...


In [49]:
articles['clean'] = articles['article'].apply(basic_clean)

In [50]:
articles.head(3)

,title,article,clean
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,may is traditionally known as asian american a...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,women in tech panelist spotlight magdalena ra...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...,women in tech panelist spotlight rachel robbi...


In [51]:
articles['stemmed'] = articles['clean'].apply(stem)

In [52]:
articles.head(3)

,title,article,clean,stemmed
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,may is traditionally known as asian american a...,may is tradit known as asian american and paci...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,women in tech panelist spotlight magdalena ra...,women in tech panelist spotlight magdalena rah...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...,women in tech panelist spotlight rachel robbi...,women in tech panelist spotlight rachel robbin...


In [55]:
articles['lemmatizatized'] = articles['clean'].apply(lemmonize)

In [56]:
articles.head(3)

,title,article,clean,stemmed,lemmatizatized
0,Spotlight on APIDA Voices: Celebrating Heritag...,May is traditionally known as Asian American a...,may is traditionally known as asian american a...,may is tradit known as asian american and paci...,may is traditionally known a asian american an...
1,Women in tech: Panelist Spotlight – Magdalena ...,Women in tech: Panelist Spotlight – Magdalena ...,women in tech panelist spotlight magdalena ra...,women in tech panelist spotlight magdalena rah...,woman in tech panelist spotlight magdalena rah...
2,Women in tech: Panelist Spotlight – Rachel Rob...,Women in tech: Panelist Spotlight – Rachel Rob...,women in tech panelist spotlight rachel robbi...,women in tech panelist spotlight rachel robbin...,woman in tech panelist spotlight rachel robbin...



For each dataframe, produce the following columns:

- title to hold the title
- original to hold the original article/post content
- clean to hold the normalized and tokenized original with the stopwords removed.
- stemmed to hold the stemmed version of the cleaned data.
- lemmatized to hold the lemmatized version of the cleaned data.
